# HW 10 CLUSTERING BUSINESS

# cluster time trends in NYC businesses: 
# DATA
# Census Business data:
## download census data for businesses by ZIP code. the data is here
http://www.census.gov/econ/cbp/download/
## and it can be downloaded by hand. you can also download it with 3 terminal commands as follows: the data from 1993 through 2001 is different in the format of its path than the data after 2001 (that is why more than one for loop is needed)



$ for ((y=93; y<=99; y+=1)); do wget ftp://ftp.census.gov/Econ2001_And_Earlier/CBP_CSV/zbp$y\totals.zip; done

$ for ((y=0; y<=9; y+=1)); do wget ftp://ftp.census.gov/econ200$y\/CBP_CSV/zbp0$y\totals.zip; done

$ for ((y=10; y<=15; y+=1)); do wget ftp://ftp.census.gov/econ20$y\/CBP_CSV/zbp$y\totals.zip; done


# NYC zip codes shape file is here
## http://data.nycprepared.org/dataset/nyc-zip-code-tabulation-areas/resource/0c0e14e9-78e1-404e-97b0-c2fabceb3981


NOTES: 
to read in a zip file without unzipping it you can use the pandas and zipfile packages

import pandas as pd

import zipfile  #i am not giving it a name cause i intend to use it only once

zf = zipfile.ZipFile('zbp13totals.zip')
df = pd.read_csv(zf.open('zbp1ztotals.zip'.replace('.zip','.txt')))



you may need to clean your data: for some NYC zip codes there may be no info
    
sanity check: you should have 20 (Ntimestamps) datapoints per time series and about 250 zipcodes (Nzipcodes)

IMPORTANT: we talked about the importance of "whitening" your data: dividing each feature by its standard deviation. 
Whitenings decorrelates the data: it makes the features independent so that the data covariance matrix is the identity matrix.
Whitening your data in time series analysis is in most cases **wrong**: you are modifying your time behaviour. This is because of the strong correlation between features (two consecutive time stamps for the same observation, the same zip code here, are strongly correlated). Here instead you want to standardize your time series: subtract the mean and divide each time series (separately) by its standard deviation. As a sanity check (if you use skitlearn Kmeans or skitlearns kmeans2): you want your data array to be shaped Nzipcodes x Ntimestamps

mydata.shape should be (Nzipcodes, Ntimestamps)

mydata[i].std() shoould be 1 for all i in range(len(Nzipcodes))

mydata[i].mean() should be ~0 for all i in range(len(Nzipcodes))



TASKS:
    
    1. get and prep your data.
    2. cluster the NUMBER OF ESTABLISHMENTS time series with K-means in **a few** clusters (as discussed there is no real good, sound way to decide what a good number is here. try a few options, keeping in mind a few is more than a couple, but i recommand you stay within the single digit numbers)
    3. plot the cluster centers (if you used K means those are the means of the clusters). you can plot for example the cluster centers overlayed on each time series (using the alpha channel to control the opacity in the plot may be helpful here).
    4. Use another clustering algorithm (of your choice)
    5. overlay your data on a NYC map: you can use shapefiles for the zip codes and different colors for different clusters
    6. Compare the results of the 2 algorithms
    7. attempt an interpretation. this is dangerous ground: clustering is an exploratory tool so you do not want to jump to conclusions because you see some clusters! but seeing structure in your data can inform your next moves as an investigator. 
    

In [43]:
import pylab as pl
import numpy as np
import pandas as pd
import json
import urllib2
import matplotlib.pyplot as plt
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## 1. Get and Prepare Data

In [315]:
import zipfile  #i am not giving it a name cause i intend to use it only once

zf = zipfile.ZipFile('zbp98totals.zip')
df = pd.read_csv(zf.open('zbp98totals.zip'.replace('.zip','.txt')))

In [348]:
# count the zipcode in nyc

url = "https://nycdatastables.s3.amazonaws.com/2013-08-19T18:18:28.877Z/nyc-zip-code-tabulation-areas-polygons.geojson"
request = urllib2.urlopen(url)
nyc = json.load(request)
# nyc
zipcode = []
for i in nyc['features']:
    zipcode.append(int(i['properties']['postalCode']))
    
len(zipcode)

262

In [349]:
yearlist = [str(i)[-2:] for i in (range(1994,2014))]
# yearlist

In [341]:
# for 94-97 and 03-13
def extractdata(year):
    zf = zipfile.ZipFile('zbp%stotals.zip'%(year))
    df = pd.read_csv(zf.open(('zbp%stotals.zip'%(year)).replace('.zip','.txt')))
    df = df[df['zip'].isin(zipcode)].loc[:,['zip','est']].set_index('zip')
    df.columns=['%s'%(year)]
    return df

# for 98-02 with uppercase
def Extractdata(year):
    zf = zipfile.ZipFile('zbp%stotals.zip'%(year))
    df = pd.read_csv(zf.open(('zbp%stotals.zip'%(year)).replace('.zip','.txt')))
    df = df[df['ZIP'].isin(zipcode)].loc[:,['ZIP','EST']].set_index('ZIP')
    df.columns=['%s'%(year)]
    return df

In [351]:
#
dfin = pd.DataFrame(index=zipcode)
for i in yearlist[:4]:
    dfin = pd.merge(dfin, extractdata(i), left_index=True, right_index=True)
for i in yearlist[4:9]:
    dfin = pd.merge(dfin, Extractdata(i), left_index=True, right_index=True)
for i in yearlist[9:]:
    dfin = pd.merge(dfin, extractdata(i), left_index=True, right_index=True)
# dfin

In [361]:
finaldata = dfin[dfin > 0]
finaldata = finaldata.dropna(how='any')
finaldata = finaldata.drop_duplicates()
print finaldata.shape
finaldata

(230, 20)


,94,95,96,97,98,99,00,01,02,03,04,05,06,07,08,09,10,11,12,13
10001,6999,7088,7075,7154,7254,7233,7317,7188,7141,7084,7370,7312,7415,7549,7472,7305,7241,7227,7235,7273
10002,1784,1819,1919,2049,2003,2085,2277,2543,2269,2402,2551,2682,2686,2685,2691,2775,2872,2912,2954,2988
10003,3256,3377,3417,3535,3649,3659,3683,3602,3616,3704,3776,3867,3909,4049,4071,4099,4113,4159,4214,4277
10004,1115,1124,1133,1173,1176,1233,1294,1247,1253,1252,1283,1260,1311,1370,1376,1379,1398,1404,1442,1479
10005,1076,1109,1216,1227,1270,1242,1287,1249,1244,1200,1226,1214,1259,1273,1291,1274,1266,1301,1273,1296
10006,1061,1002,969,990,1030,1059,1067,1026,934,862,869,870,874,863,845,828,843,834,850,852
10007,1307,1274,1254,1287,1282,1303,1327,1233,1620,1603,1635,1311,1292,1284,1313,1277,1330,1368,1400,1424
10009,634,633,673,747,786,804,860,869,921,953,982,1029,1016,1027,1066,1049,1090,1128,1176,1220
10010,3019,2987,3029,3112,3152,3143,3203,3178,3085,3045,3059,3060,3047,2948,3000,2938,2981,3022,3068,3091
10011,3301,3340,3392,3541,3599,3664,3653,3592,3682,3711,3729,3784,3823,3818,3887,3823,3852,3869,3841,3905


In [363]:
# "whitening" the data
newdata = finaldata.copy()
for i in range(len(finaldata)):
    newdata.iloc[i] = (newdata.iloc[i]-np.mean(newdata.iloc[i]))/np.std(newdata.iloc[i])
newdata

,94,95,96,97,98,99,00,01,02,03,04,05,06,07,08,09,10,11,12,13
10001,-1.831632,-1.173250,-1.269418,-0.685012,0.054742,-0.100607,0.520787,-0.433496,-0.781180,-1.202840,0.912857,0.483799,1.245746,2.237017,1.667406,0.432016,-0.041426,-0.144992,-0.085811,0.195295
10002,-1.722813,-1.631907,-1.372173,-1.034519,-1.153997,-0.941015,-0.442326,0.248565,-0.463105,-0.117659,0.269344,0.609595,0.619984,0.617387,0.632971,0.851147,1.103089,1.206982,1.316071,1.404380
10003,-1.872528,-1.457250,-1.319968,-0.914985,-0.523731,-0.489411,-0.407041,-0.685038,-0.636989,-0.334968,-0.087861,0.224456,0.368602,0.849090,0.924595,1.020693,1.068741,1.226616,1.415379,1.631598
10004,-1.608897,-1.523770,-1.438643,-1.060302,-1.031926,-0.492790,0.084181,-0.360370,-0.303619,-0.313078,-0.019863,-0.237409,0.244976,0.803030,0.859781,0.888157,1.067869,1.124620,1.484044,1.834010
10005,-2.885957,-2.304004,-0.417066,-0.223082,0.535220,0.041442,0.835014,0.164887,0.076712,-0.699225,-0.240717,-0.452336,0.341236,0.588125,0.905554,0.605760,0.464681,1.081903,0.588125,0.993729
10006,1.563021,0.877893,0.494686,0.738545,1.203039,1.539797,1.632695,1.156589,0.088254,-0.747835,-0.666548,-0.654936,-0.608487,-0.736222,-0.945245,-1.142654,-0.968469,-1.072980,-0.887183,-0.863958
10007,-0.412975,-0.689971,-0.857847,-0.580851,-0.622820,-0.446550,-0.245099,-1.034117,2.214286,2.071591,2.340193,-0.379400,-0.538882,-0.606033,-0.362612,-0.664789,-0.219918,0.099047,0.367649,0.569100
10009,-1.738195,-1.744005,-1.511588,-1.081615,-0.855007,-0.750419,-0.425034,-0.372740,-0.070597,0.115337,0.283840,0.556931,0.481395,0.545310,0.771918,0.673140,0.911368,1.132165,1.411067,1.666726
10010,-0.553445,-1.003514,-0.412798,0.754570,1.317157,1.190575,2.034455,1.682838,0.374824,-0.187763,0.009142,0.023207,-0.159634,-1.552037,-0.820674,-1.692683,-1.087902,-0.511251,0.135724,0.459212
10011,-2.187392,-1.968260,-1.676083,-0.838884,-0.512995,-0.147774,-0.209581,-0.552326,-0.046636,0.116309,0.217447,0.526480,0.745613,0.717519,1.105215,0.745613,0.908557,1.004077,0.846751,1.206353


## 2. Cluster (k-mean)